# С помощью бертов решить задачу классификации

## Импорты

In [25]:
import numpy as np
import pandas as pd

import torch
from torch import argmax
from torch.nn.functional import softmax
from transformers import BertTokenizerFast, AutoModelForSequenceClassification


## Настройки

In [26]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL = 'blanchefort/rubert-base-cased-sentiment'


In [27]:
DEVICE

device(type='cuda', index=0)

## Модель

In [28]:
class SentimentClassifier:
    CALSSES = ['Нейтральный', 'Позитивный', 'Негативный']

    def __init__(self, model, device='cpu'):
        self._tokenizer = BertTokenizerFast.from_pretrained(model)
        self._model = AutoModelForSequenceClassification.from_pretrained(model)
        self.to(device)

    @torch.no_grad()
    def predict(self, text, aggregate=True):
        inputs = self._tokenizer(text,
                                 max_length=512,
                                 padding=True,
                                 truncation=True,
                                 return_tensors='pt')

        inputs = inputs.to(self._device)
        outputs = self._model(**inputs)
        predicted = softmax(outputs.logits, dim=1)
        predicted = argmax(predicted, dim=1).to('cpu').numpy()

        if aggregate:
            try:
                return self.CALSSES[predicted[0]]
            except:
                return 'Ошибка определения класса!'

        return predicted

    def to(self, device):
        self._model = self._model.to(device)
        self._device = device

classifier = SentimentClassifier(MODEL, device=device)

In [29]:
print(classifier.predict('Огни на главной новогодней елке на Дворцовой площади зажгут 20 декабря'))

Нейтральный


In [30]:
print(classifier.predict('Из Петербурга в Донецкую Народную Республику была отправлена тонна гуманитарного груза'))

Нейтральный


In [31]:
print(classifier.predict('Выросло число пострадавших при стрельбе в Брянске'))

Негативный


In [32]:
print(classifier.predict('В Подмосковье нашли тело убитого экс-депутата Верховной рады Ильи Кивы  '))

Нейтральный


In [33]:
print(classifier.predict('Сегодня уроков не будет'))

Негативный


In [34]:
print(classifier.predict('Сегодня будет солнечный день'))

Позитивный
